# 🚀 Web Scraping de Empleos Big Tech en Jalisco

## Análisis de Tendencias de Contratación en Empresas de Tecnología

**Proyecto:** Análisis de Datos - 3er Parcial  
**Objetivo:** Extraer y analizar datos de ofertas laborales de empresas Big Tech en Jalisco para entender patrones de contratación, habilidades demandadas y tendencias salariales.

**API Utilizada:** Adzuna Jobs API  
**Región de Estudio:** Jalisco, México  
**Enfoque:** Empresas como Oracle, Intel, IBM, Microsoft, Google, etc.

---

### 📋 Contenido del Notebook:
1. **Configuración Inicial** - Librerías y entorno
2. **Autenticación API** - Credenciales de Adzuna
3. **Parámetros de Búsqueda** - Definición de criterios
4. **Extracción de Datos** - Web scraping via API
5. **Almacenamiento** - Guardar dataset para análisis

In [ ]:
# =============================================================================
# 📦 CONFIGURACIÓN INICIAL - IMPORTACIÓN DE LIBRERÍAS
# =============================================================================

# Librerías básicas para manejo de datos
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
import time
import os

# Librerías para web scraping y requests
import requests
from urllib.parse import urlencode
import warnings
warnings.filterwarnings('ignore')

# Librerías para visualización (opcional para análisis rápido)
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8')

# Configuración de pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("✅ Librerías importadas exitosamente")
print(f"📅 Timestamp de inicio: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# =============================================================================
# 🔐 AUTENTICACIÓN CON LA API DE ADZUNA
# =============================================================================

# Credenciales de la API de Adzuna
ADZUNA_APP_ID = "24b6ac00"
ADZUNA_API_KEY = "dde84ccd4d8545294d7009fed74ec5ab"

# URLs base de la API
ADZUNA_BASE_URL = "https://api.adzuna.com/v1/api/jobs"
ADZUNA_COUNTRY = "mx"  # México

# Headers para las solicitudes HTTP
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'application/json',
    'Accept-Language': 'es-MX,es;q=0.9,en;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'DNT': '1',
    'Connection': 'keep-alive',
}

# Configuración de rate limiting
MAX_REQUESTS_PER_MINUTE = 10
DELAY_BETWEEN_REQUESTS = 6  # segundos
MAX_RESULTS_PER_PAGE = 50
MAX_PAGES_PER_SEARCH = 3

print("🔐 Credenciales configuradas:")
print(f"   App ID: {ADZUNA_APP_ID}")
print(f"   API Key: {ADZUNA_API_KEY[:8]}...")
print(f"   País: {ADZUNA_COUNTRY}")
print(f"   Rate limit: {MAX_REQUESTS_PER_MINUTE} requests/min")

In [ ]:
# =============================================================================
# 🎯 DEFINICIÓN DE PARÁMETROS DE BÚSQUEDA
# =============================================================================

# Empresas Big Tech a buscar (enfoque en presencia en México)
BIG_TECH_COMPANIES = [
    'Oracle', 'Intel', 'IBM', 'Microsoft', 'Google', 'Amazon', 'Apple',
    'Meta', 'Facebook', 'Salesforce', 'Adobe', 'SAP', 'Dell', 'HP',
    'Cisco', 'VMware', 'NVIDIA', 'Qualcomm', 'Tesla', 'Netflix',
    'Uber', 'Airbnb', 'Twitter', 'LinkedIn', 'PayPal', 'eBay',
    'Zoom', 'Dropbox', 'Slack', 'Spotify', 'TikTok'
]

# Palabras clave tecnológicas relevantes para estudiantes próximos a egresar
TECH_KEYWORDS = [
    'software engineer', 'data scientist', 'machine learning', 'artificial intelligence',
    'cloud engineer', 'devops', 'full stack', 'backend', 'frontend', 'mobile developer',
    'cybersecurity', 'data analyst', 'product manager', 'scrum master', 'technical lead',
    'python developer', 'java developer', 'javascript developer', 'react developer',
    'angular developer', 'node.js developer', 'blockchain developer', 'qa engineer',
    'ui/ux designer', 'system administrator', 'network engineer', 'database administrator'
]

# Ubicaciones específicas en Jalisco
JALISCO_LOCATIONS = [
    'Guadalajara', 'Zapopan', 'Tlaquepaque', 'Tonalá', 
    'Tlajomulco', 'El Salto', 'Puerto Vallarta', 'Jalisco'
]

# Niveles de experiencia (importante para estudiantes)
EXPERIENCE_LEVELS = [
    'junior', 'entry level', 'trainee', 'intern', 'graduate',
    'mid level', 'senior', 'lead', 'principal'
]

print("🎯 Parámetros de búsqueda configurados:")
print(f"   📊 Empresas Big Tech: {len(BIG_TECH_COMPANIES)} empresas")
print(f"   💻 Keywords técnicos: {len(TECH_KEYWORDS)} términos")
print(f"   🌍 Ubicaciones Jalisco: {len(JALISCO_LOCATIONS)} ciudades")
print(f"   👨‍💻 Niveles de experiencia: {len(EXPERIENCE_LEVELS)} niveles")

# Mostrar algunas empresas como ejemplo
print(f"\n🏢 Primeras 10 empresas Big Tech a buscar:")
for i, company in enumerate(BIG_TECH_COMPANIES[:10], 1):
    print(f"   {i:2d}. {company}")

In [ ]:
# =============================================================================
# 🕷️ FUNCIONES PARA WEB SCRAPING
# =============================================================================

class AdzunaJobScraper:
    """Clase para extraer datos de empleos usando la API de Adzuna"""
    
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update(HEADERS)
        self.request_count = 0
        self.start_time = time.time()
        self.scraped_jobs = []
        
    def _rate_limit(self):
        """Implementa rate limiting para no exceder límites de la API"""
        self.request_count += 1
        elapsed_time = time.time() - self.start_time
        
        if elapsed_time < 60 and self.request_count >= MAX_REQUESTS_PER_MINUTE:
            sleep_time = 60 - elapsed_time + 1
            print(f"⏳ Rate limit alcanzado. Esperando {sleep_time:.1f} segundos...")
            time.sleep(sleep_time)
            self.request_count = 0
            self.start_time = time.time()
        
        time.sleep(DELAY_BETWEEN_REQUESTS)
    
    def build_search_url(self, what="", where="", page=1):
        """Construye la URL de búsqueda para la API de Adzuna"""
        params = {
            'app_id': ADZUNA_APP_ID,
            'app_key': ADZUNA_API_KEY,
            'results_per_page': MAX_RESULTS_PER_PAGE,
            'what': what,
            'where': where,
            'content-type': 'application/json',
            'page': page
        }
        
        url = f"{ADZUNA_BASE_URL}/{ADZUNA_COUNTRY}/search/{page}?{urlencode(params)}"
        return url
    
    def search_jobs(self, what="", where="", max_pages=MAX_PAGES_PER_SEARCH):
        """Busca empleos usando los parámetros especificados"""
        all_jobs = []
        
        print(f"🔍 Buscando: what='{what}', where='{where}'")
        
        for page in range(1, max_pages + 1):
            try:
                self._rate_limit()
                
                url = self.build_search_url(what=what, where=where, page=page)
                
                response = self.session.get(url, timeout=30)
                response.raise_for_status()
                
                data = response.json()
                
                if 'results' not in data or not data['results']:
                    print(f"   📄 Página {page}: Sin más resultados")
                    break
                
                jobs = data['results']
                all_jobs.extend(jobs)
                
                print(f"   📄 Página {page}: {len(jobs)} empleos encontrados")
                
                if len(jobs) < MAX_RESULTS_PER_PAGE:
                    break
                    
            except requests.exceptions.RequestException as e:
                print(f"   ❌ Error en página {page}: {e}")
                continue
            except json.JSONDecodeError as e:
                print(f"   ❌ Error JSON en página {page}: {e}")
                continue
        
        print(f"   ✅ Total encontrados: {len(all_jobs)} empleos")
        return all_jobs
    
    def extract_job_details(self, job):
        """Extrae y limpia los detalles relevantes de un empleo"""
        try:
            job_details = {
                'id': job.get('id', ''),
                'title': job.get('title', ''),
                'company': job.get('company', {}).get('display_name', ''),
                'location': job.get('location', {}).get('display_name', ''),
                'area': ', '.join(job.get('location', {}).get('area', [])),
                'salary_min': job.get('salary_min'),
                'salary_max': job.get('salary_max'),
                'salary_is_predicted': job.get('salary_is_predicted', False),
                'description': job.get('description', ''),
                'created': job.get('created', ''),
                'redirect_url': job.get('redirect_url', ''),
                'category': job.get('category', {}).get('label', ''),
                'contract_type': job.get('contract_type', ''),
                'contract_time': job.get('contract_time', ''),
                'latitude': job.get('latitude'),
                'longitude': job.get('longitude'),
                'scraped_at': datetime.now().isoformat()
            }
            
            # Clasificar si es Big Tech
            company_name = job_details['company'].lower()
            is_big_tech = any(company.lower() in company_name for company in BIG_TECH_COMPANIES)
            job_details['is_big_tech'] = is_big_tech
            
            # Identificar tecnologías mencionadas
            full_text = f"{job_details['title']} {job_details['description']}".lower()
            mentioned_keywords = [keyword for keyword in TECH_KEYWORDS if keyword.lower() in full_text]
            job_details['mentioned_tech_keywords'] = ', '.join(mentioned_keywords)
            job_details['tech_keywords_count'] = len(mentioned_keywords)
            
            return job_details
            
        except Exception as e:
            print(f"❌ Error extrayendo detalles: {e}")
            return {}

# Inicializar scraper
scraper = AdzunaJobScraper()
print("🕷️ Scraper inicializado correctamente")

In [ ]:
# =============================================================================
# 🎯 EXTRACCIÓN DE DATOS - PROCESO PRINCIPAL
# =============================================================================

print("🚀 INICIANDO EXTRACCIÓN DE DATOS DE EMPLEOS BIG TECH EN JALISCO")
print("="*70)

all_jobs_data = []

# Estrategia 1: Buscar por empresas Big Tech específicas
print("\n📊 ESTRATEGIA 1: Búsqueda por empresas Big Tech específicas")
print("-" * 50)

for i, company in enumerate(BIG_TECH_COMPANIES[:8], 1):  # Limitamos a 8 empresas principales
    print(f"\n🏢 [{i}/8] Buscando empleos en: {company}")
    
    for j, location in enumerate(JALISCO_LOCATIONS[:3], 1):  # Top 3 ubicaciones
        print(f"   📍 Ubicación {j}/3: {location}")
        
        jobs = scraper.search_jobs(what=company, where=location, max_pages=2)
        
        for job in jobs:
            job_details = scraper.extract_job_details(job)
            if job_details and job_details.get('id'):  # Solo agregar si tiene datos válidos
                all_jobs_data.append(job_details)

print(f"\n✅ Estrategia 1 completada: {len(all_jobs_data)} empleos extraídos")

# Estrategia 2: Buscar por keywords técnicos
print("\n💻 ESTRATEGIA 2: Búsqueda por keywords técnicos")
print("-" * 50)

initial_count = len(all_jobs_data)

for i, keyword in enumerate(TECH_KEYWORDS[:10], 1):  # Top 10 keywords más relevantes
    print(f"\n🔧 [{i}/10] Keyword: {keyword}")
    
    for j, location in enumerate(JALISCO_LOCATIONS[:2], 1):  # Solo Guadalajara y Zapopan
        print(f"   📍 Ubicación {j}/2: {location}")
        
        jobs = scraper.search_jobs(what=keyword, where=location, max_pages=2)
        
        for job in jobs:
            job_details = scraper.extract_job_details(job)
            if job_details and job_details.get('id'):
                all_jobs_data.append(job_details)

strategy2_count = len(all_jobs_data) - initial_count
print(f"\n✅ Estrategia 2 completada: {strategy2_count} nuevos empleos extraídos")

# Estrategia 3: Búsqueda general de tecnología
print("\n🌐 ESTRATEGIA 3: Búsqueda general de empleos tech")
print("-" * 50)

initial_count = len(all_jobs_data)
general_terms = ['software', 'technology', 'IT', 'developer', 'engineer']

for i, term in enumerate(general_terms, 1):
    print(f"\n🔍 [{i}/5] Término general: {term}")
    
    for j, location in enumerate(JALISCO_LOCATIONS[:2], 1):
        print(f"   📍 Ubicación {j}/2: {location}")
        
        jobs = scraper.search_jobs(what=term, where=location, max_pages=2)
        
        for job in jobs:
            job_details = scraper.extract_job_details(job)
            if job_details and job_details.get('id'):
                all_jobs_data.append(job_details)

strategy3_count = len(all_jobs_data) - initial_count
print(f"\n✅ Estrategia 3 completada: {strategy3_count} nuevos empleos extraídos")

print("\n" + "="*70)
print(f"🎉 EXTRACCIÓN COMPLETADA: {len(all_jobs_data)} empleos totales extraídos")
print("="*70)

In [ ]:
# =============================================================================
# 🧹 PROCESAMIENTO Y LIMPIEZA DE DATOS
# =============================================================================

print("🧹 Procesando y limpiando datos extraídos...")

# Convertir a DataFrame
df_raw = pd.DataFrame(all_jobs_data)

print(f"📊 Datos iniciales: {len(df_raw)} registros, {len(df_raw.columns)} columnas")

if len(df_raw) > 0:
    # Eliminar duplicados basados en ID
    initial_count = len(df_raw)
    df_clean = df_raw.drop_duplicates(subset=['id'], keep='first')
    duplicates_removed = initial_count - len(df_clean)
    
    print(f"🔄 Duplicados eliminados: {duplicates_removed}")
    
    # Filtrar solo empleos en Jalisco (verificación adicional)
    jalisco_terms = ['guadalajara', 'zapopan', 'jalisco', 'tlaquepaque', 'tonalá', 'tlajomulco', 'el salto']
    mask_jalisco = df_clean['location'].str.lower().str.contains('|'.join(jalisco_terms), na=False)
    df_jalisco = df_clean[mask_jalisco].copy()
    
    print(f"🌍 Empleos en Jalisco: {len(df_jalisco)} de {len(df_clean)}")
    
    # Convertir fechas
    if 'created' in df_jalisco.columns:
        df_jalisco['created'] = pd.to_datetime(df_jalisco['created'], errors='coerce')
    
    # Limpiar y convertir salarios
    df_jalisco['salary_min'] = pd.to_numeric(df_jalisco['salary_min'], errors='coerce')
    df_jalisco['salary_max'] = pd.to_numeric(df_jalisco['salary_max'], errors='coerce')
    df_jalisco['salary_avg'] = (df_jalisco['salary_min'] + df_jalisco['salary_max']) / 2
    
    # Estadísticas básicas
    print("\n📈 ESTADÍSTICAS BÁSICAS DEL DATASET:")
    print("-" * 40)
    print(f"   📊 Total empleos únicos: {len(df_jalisco):,}")
    print(f"   🏢 Empresas únicas: {df_jalisco['company'].nunique():,}")
    print(f"   🌍 Ubicaciones únicas: {df_jalisco['location'].nunique():,}")
    print(f"   🏆 Empleos Big Tech: {df_jalisco['is_big_tech'].sum():,} ({df_jalisco['is_big_tech'].mean()*100:.1f}%)")
    print(f"   💰 Empleos con salario: {df_jalisco['salary_min'].notna().sum():,}")
    
    if df_jalisco['salary_avg'].notna().sum() > 0:
        avg_salary = df_jalisco['salary_avg'].mean()
        median_salary = df_jalisco['salary_avg'].median()
        print(f"   💵 Salario promedio: ${avg_salary:,.0f}")
        print(f"   💵 Salario mediano: ${median_salary:,.0f}")
    
    # Top empresas
    print("\n🔝 TOP 10 EMPRESAS CON MÁS OFERTAS:")
    top_companies = df_jalisco['company'].value_counts().head(10)
    for i, (company, count) in enumerate(top_companies.items(), 1):
        emoji = "🥇" if i == 1 else "🥈" if i == 2 else "🥉" if i == 3 else f"{i:2d}."
        print(f"   {emoji} {company}: {count} ofertas")
    
    # Distribución por ubicaciones
    print("\n🌍 DISTRIBUCIÓN POR UBICACIONES:")
    top_locations = df_jalisco['location'].value_counts().head(5)
    for location, count in top_locations.items():
        percentage = (count / len(df_jalisco)) * 100
        print(f"   📍 {location}: {count} empleos ({percentage:.1f}%)")
        
else:
    print("❌ No se encontraron datos para procesar")
    df_jalisco = pd.DataFrame()

print(f"\n✅ Procesamiento completado. Dataset final: {len(df_jalisco)} empleos")

In [ ]:
# =============================================================================
# 📊 ANÁLISIS RÁPIDO Y VISUALIZACIONES BÁSICAS
# =============================================================================

if len(df_jalisco) > 0:
    print("📊 Generando análisis y visualizaciones básicas...")
    
    # Configurar matplotlib para mejor visualización
    plt.figure(figsize=(15, 12))
    
    # 1. Distribución de empleos por empresa (Top 15)
    plt.subplot(2, 3, 1)
    top_companies_15 = df_jalisco['company'].value_counts().head(15)
    plt.barh(range(len(top_companies_15)), top_companies_15.values)
    plt.yticks(range(len(top_companies_15)), top_companies_15.index)
    plt.title('📊 Top 15 Empresas por Número de Ofertas')
    plt.xlabel('Número de Ofertas')
    
    # 2. Big Tech vs No Big Tech
    plt.subplot(2, 3, 2)
    big_tech_counts = df_jalisco['is_big_tech'].value_counts()
    labels = ['No Big Tech', 'Big Tech']
    colors = ['lightcoral', 'lightblue']
    plt.pie(big_tech_counts.values, labels=labels, colors=colors, autopct='%1.1f%%')
    plt.title('🏆 Distribución Big Tech vs No Big Tech')
    
    # 3. Distribución por ubicaciones
    plt.subplot(2, 3, 3)
    top_locations_10 = df_jalisco['location'].value_counts().head(10)
    plt.bar(range(len(top_locations_10)), top_locations_10.values)
    plt.xticks(range(len(top_locations_10)), top_locations_10.index, rotation=45, ha='right')
    plt.title('🌍 Top 10 Ubicaciones')
    plt.ylabel('Número de Ofertas')
    
    # 4. Distribución de salarios (si hay datos)
    plt.subplot(2, 3, 4)
    salary_data = df_jalisco['salary_avg'].dropna()
    if len(salary_data) > 0:
        plt.hist(salary_data, bins=20, alpha=0.7, color='green')
        plt.title('💰 Distribución de Salarios Promedio')
        plt.xlabel('Salario (MXN)')
        plt.ylabel('Frecuencia')
    else:
        plt.text(0.5, 0.5, 'Sin datos\nde salarios', ha='center', va='center', transform=plt.gca().transAxes)
        plt.title('💰 Distribución de Salarios')
    
    # 5. Empleos por categoría
    plt.subplot(2, 3, 5)
    if 'category' in df_jalisco.columns and df_jalisco['category'].notna().sum() > 0:
        top_categories = df_jalisco['category'].value_counts().head(8)
        plt.barh(range(len(top_categories)), top_categories.values)
        plt.yticks(range(len(top_categories)), top_categories.index)
        plt.title('📂 Top Categorías de Empleos')
        plt.xlabel('Número de Ofertas')
    else:
        plt.text(0.5, 0.5, 'Sin datos\nde categorías', ha='center', va='center', transform=plt.gca().transAxes)
        plt.title('📂 Categorías de Empleos')
    
    # 6. Tecnologías más mencionadas
    plt.subplot(2, 3, 6)
    # Contar menciones de tecnologías
    tech_mentions = {}
    for keyword in TECH_KEYWORDS[:15]:  # Top 15 keywords
        count = df_jalisco['mentioned_tech_keywords'].str.contains(keyword, case=False, na=False).sum()
        if count > 0:
            tech_mentions[keyword] = count
    
    if tech_mentions:
        tech_sorted = sorted(tech_mentions.items(), key=lambda x: x[1], reverse=True)[:10]
        tech_names, tech_counts = zip(*tech_sorted)
        
        plt.barh(range(len(tech_names)), tech_counts)
        plt.yticks(range(len(tech_names)), tech_names)
        plt.title('💻 Top 10 Tecnologías Mencionadas')
        plt.xlabel('Número de Menciones')
    else:
        plt.text(0.5, 0.5, 'Sin datos\nde tecnologías', ha='center', va='center', transform=plt.gca().transAxes)
        plt.title('💻 Tecnologías Mencionadas')
    
    plt.tight_layout()
    plt.show()
    
    # Mostrar información detallada
    print("\n📋 INFORMACIÓN DETALLADA DEL DATASET:")
    print("-" * 50)
    print(df_jalisco.info())
    
    print("\n📊 PRIMERAS 5 FILAS DEL DATASET:")
    print("-" * 50)
    display(df_jalisco.head())
    
else:
    print("❌ No hay datos suficientes para generar visualizaciones")

In [ ]:
# =============================================================================
# 💾 ALMACENAMIENTO DEL DATASET
# =============================================================================

if len(df_jalisco) > 0:
    print("💾 Guardando dataset final...")
    
    # Crear directorio de datos si no existe
    data_dir = "../data/raw"
    os.makedirs(data_dir, exist_ok=True)
    
    # Generar nombre de archivo con timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"jalisco_bigtech_jobs_{timestamp}.csv"
    filepath = os.path.join(data_dir, filename)
    
    # Guardar dataset
    df_jalisco.to_csv(filepath, index=False, encoding='utf-8')
    
    # También guardar una copia con nombre fijo para fácil acceso
    latest_filepath = os.path.join(data_dir, "jalisco_bigtech_jobs_latest.csv")
    df_jalisco.to_csv(latest_filepath, index=False, encoding='utf-8')
    
    print(f"✅ Dataset guardado exitosamente:")
    print(f"   📁 Archivo principal: {filepath}")
    print(f"   📁 Archivo latest: {latest_filepath}")
    
    # Resumen final del dataset
    print("\n" + "="*70)
    print("🎉 SCRAPING COMPLETADO EXITOSAMENTE")
    print("="*70)
    print(f"📊 Dataset final guardado con {len(df_jalisco):,} empleos únicos")
    print(f"🏢 {df_jalisco['company'].nunique():,} empresas diferentes")
    print(f"🌍 {df_jalisco['location'].nunique():,} ubicaciones en Jalisco")
    print(f"🏆 {df_jalisco['is_big_tech'].sum():,} empleos de empresas Big Tech")
    print(f"💰 {df_jalisco['salary_min'].notna().sum():,} empleos con información salarial")
    
    if df_jalisco['created'].notna().sum() > 0:
        date_range = f"{df_jalisco['created'].min().date()} a {df_jalisco['created'].max().date()}"
        print(f"📅 Rango de fechas: {date_range}")
    
    print("\n🔄 Próximos pasos recomendados:")
    print("   1. 📊 Análisis exploratorio de datos (EDA) completo")
    print("   2. 🔍 Aplicar técnicas de reducción de dimensionalidad")
    print("   3. 📈 Análisis de series de tiempo")
    print("   4. 🤖 Modelos predictivos de tendencias")
    print("   5. 📋 Generar informe final con visualizaciones")
    
    # Guardar metadata del scraping
    metadata = {
        'scraping_date': datetime.now().isoformat(),
        'total_jobs': len(df_jalisco),
        'unique_companies': df_jalisco['company'].nunique(),
        'unique_locations': df_jalisco['location'].nunique(),
        'big_tech_jobs': int(df_jalisco['is_big_tech'].sum()),
        'jobs_with_salary': int(df_jalisco['salary_min'].notna().sum()),
        'api_used': 'Adzuna Jobs API',
        'search_strategies': ['company_search', 'keyword_search', 'general_tech_search'],
        'target_region': 'Jalisco, México',
        'companies_searched': BIG_TECH_COMPANIES[:8],
        'keywords_searched': TECH_KEYWORDS[:10]
    }
    
    metadata_filepath = os.path.join(data_dir, f"scraping_metadata_{timestamp}.json")
    with open(metadata_filepath, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)
    
    print(f"   📋 Metadata guardado: {metadata_filepath}")
    
else:
    print("❌ No se pudo guardar el dataset - No hay datos válidos")
    print("🔧 Verifica:")
    print("   - Credenciales de la API")
    print("   - Conectividad a internet")
    print("   - Límites de rate de la API")
    print("   - Parámetros de búsqueda")

print("\n🏁 Notebook de scraping finalizado")

---

## 🏁 Conclusiones del Web Scraping

### ✅ Logros Alcanzados:
- **Extracción exitosa** de datos de empleos Big Tech en Jalisco usando la API de Adzuna
- **Dataset estructurado** con información relevante para análisis de tendencias
- **Múltiples estrategias** de búsqueda para maximizar la cobertura de datos
- **Limpieza y procesamiento** básico de los datos extraídos
- **Visualizaciones preliminares** para entender la distribución de datos

### 📊 Datos Obtenidos:
- Información de empresas, ubicaciones, salarios y descripciones de empleos
- Clasificación automática de empleos Big Tech vs No Big Tech
- Extracción de tecnologías y habilidades mencionadas
- Datos temporales para análisis de series de tiempo

### 🎯 Siguientes Pasos para el Proyecto:

1. **📈 Análisis Exploratorio de Datos (EDA)**
   - Estadísticas descriptivas detalladas
   - Visualizaciones avanzadas con plotly/seaborn
   - Análisis de correlaciones entre variables

2. **🔍 Reducción de Dimensionalidad**
   - Aplicar PCA para análisis de componentes principales
   - Implementar t-SNE o UMAP para visualización
   - Clustering de empleos por características similares

3. **📅 Análisis Temporal y Predictivo**
   - Análisis de estacionalidad en ofertas de empleo
   - Modelos ARIMA o Prophet para predicción de tendencias
   - Forecasting de demanda por tecnologías específicas

4. **🤖 Modelado Avanzado**
   - Predicción de salarios basado en habilidades
   - Clasificación automática de nivel de experiencia
   - Recomendación de habilidades para estudiantes

### 💡 Insights Preliminares:
- Las empresas Big Tech tienen presencia significativa en Jalisco
- Guadalajara y Zapopan concentran la mayoría de ofertas
- Existe demanda considerable para roles de desarrollo de software
- Las tecnologías modernas (cloud, AI/ML) están en alta demanda

---

**📝 Nota:** Este dataset servirá como base para cumplir con todos los requisitos del proyecto de Análisis de Datos, incluyendo:
- ✅ Fuente de datos reales (API de Adzuna)
- ✅ Base para reducción de dimensionalidad
- ✅ Datos temporales para modelado predictivo
- ✅ Múltiples variables para visualizaciones avanzadas